In [ ]:
pip install pyarrow #in local machine you'll hv to install. colab already has it

importing libraries and ingesting parquet file

In [ ]:
import pyarrow.parquet as pq
trips = pq.read_table('/content/drive/MyDrive/Colab Notebooks/Data Engineering/WEEK 5/data/raw/yellow_tripdata_2025-08.parquet') #read_table reads the data into a table

In [ ]:
type(trips)

pyarrow.lib.Table

In [ ]:
trips.shape #this format is too big and cannt get head info

(3574091, 20)

In [ ]:
trips=trips.to_pandas() #converted into a pandas dataframe

In [ ]:
trips.shape #ensuring all data is there

(3574091, 20)

In [ ]:
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,1.0,N,138,141,1,33.8,6.0,0.5,5.00,6.94,1.0,57.49,2.5,1.75,0.00
1,2,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,1.0,N,138,193,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,0.0,1.75,0.00
2,7,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,1.0,N,249,45,1,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2.5,0.00,0.75
3,7,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,1.0,N,79,229,1,11.4,0.0,0.5,3.43,0.00,1.0,20.58,2.5,0.00,0.75
4,2,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,1.0,N,43,48,1,11.4,1.0,0.5,2.57,0.00,1.0,19.72,2.5,0.00,0.75


In [ ]:
df=trips.copy() #having a copy of the og trips dataframe

In [ ]:
#checking if there are duplicates
print(f'Number of duplicate rows: {df.duplicated().sum()}')

Number of duplicate rows: 0


deleting duplicates and assigning an index column which changes when the duplicates are taken out

In [ ]:
df= df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [ ]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee', 'trip_id'],
      dtype='object')

using the columns shown above u create a fact-dimension model. and then make dataframes on python for each dimension u identify.

In [ ]:
#create datetime_dim. extracting month, time, day, year from the info given.
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['datetime_id'] = datetime_dim.index

# datetime_dim = datetime_dim.rename(columns={'tpep_pickup_datetime': 'datetime_id'}).reset_index(drop=True)
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday','tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
#
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2025-08-01 00:52:23,0,1,8,2025,4,2025-08-01 01:12:20,1,1,8,2025,4
1,1,2025-08-01 00:03:01,0,1,8,2025,4,2025-08-01 00:15:33,0,1,8,2025,4
2,2,2025-08-01 00:24:38,0,1,8,2025,4,2025-08-01 00:24:38,0,1,8,2025,4
3,3,2025-08-01 00:48:19,0,1,8,2025,4,2025-08-01 00:48:19,0,1,8,2025,4
4,4,2025-08-01 00:25:34,0,1,8,2025,4,2025-08-01 00:33:18,0,1,8,2025,4


In [ ]:
#passenger count dim
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,1.0
1,1,2.0
2,2,2.0
3,3,1.0
4,4,1.0


In [ ]:
#trip distance dim
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim.head()

,trip_distance_id,trip_distance
0,0,8.44
1,1,4.98
2,2,1.89
3,3,2.35
4,4,2.14


In [ ]:
#rate code dim
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
rate_code_dim.head()


,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate


In [ ]:
#payment type dim
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]
payment_type_dim.head()


,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card


In [ ]:
#fact table
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id',right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',\
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]
fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,33.8,6.0,0.5,5.00,6.94,1.0,57.49
1,1,2,1,1,1,1,N,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45
2,2,7,2,2,2,2,N,2,14.2,0.0,0.5,3.99,0.00,1.0,23.94
3,3,7,3,3,3,3,N,3,11.4,0.0,0.5,3.43,0.00,1.0,20.58
4,4,2,4,4,4,4,N,4,11.4,1.0,0.5,2.57,0.00,1.0,19.72


load the dimension and fact tables into SQLite database

In [ ]:
#Create a connection to the SQLite database.
import sqlite3
conn = sqlite3.connect('nyc_taxi_data.db')


In [ ]:
#load each dimension into different tables in the database
datetime_dim.to_sql('datetime_dim', conn, if_exists='replace', index=False)
passenger_count_dim.to_sql('passenger_count_dim', conn, if_exists='replace', index=False)
trip_distance_dim.to_sql('trip_distance_dim', conn, if_exists='replace', index=False)
rate_code_dim.to_sql('rate_code_dim', conn, if_exists='replace', index=False)
payment_type_dim.to_sql('payment_type_dim', conn, if_exists='replace', index=False)


3574091

In [ ]:
#loading the fact table into the database
fact_table.to_sql('fact_table', conn, if_exists='replace', index=False)

3574091

reading data back from the SQLite tables to make sure the data was loaded correctly

In [ ]:
import pandas as pd

# Read data from a dimension table
dim_query = "SELECT * FROM datetime_dim LIMIT 5;"
dim_df = pd.read_sql_query(dim_query, conn)
display(dim_df.head())


,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2025-08-01 00:52:23,0,1,8,2025,4,2025-08-01 01:12:20,1,1,8,2025,4
1,1,2025-08-01 00:03:01,0,1,8,2025,4,2025-08-01 00:15:33,0,1,8,2025,4
2,2,2025-08-01 00:24:38,0,1,8,2025,4,2025-08-01 00:24:38,0,1,8,2025,4
3,3,2025-08-01 00:48:19,0,1,8,2025,4,2025-08-01 00:48:19,0,1,8,2025,4
4,4,2025-08-01 00:25:34,0,1,8,2025,4,2025-08-01 00:33:18,0,1,8,2025,4


In [ ]:
#close the connection to SQLite database
import sqlite3
conn = sqlite3.connect('nyc_taxi_data.db')

In [ ]:
sampling_query = "SELECT * FROM fact_table ORDER BY RANDOM() LIMIT 1000;"
sampled_fact_table = pd.read_sql_query(sampling_query, conn)
display(sampled_fact_table.head())

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1511848,2,1511848,1511848,1511848,1511848,N,1511848,7.90,0.0,0.5,0.00,0.0,1.0,12.65
1,1109493,2,1109493,1109493,1109493,1109493,N,1109493,14.90,2.5,0.5,4.43,0.0,1.0,26.58
2,1806539,2,1806539,1806539,1806539,1806539,N,1806539,5.80,2.5,0.5,2.61,0.0,1.0,15.66
3,2061287,2,2061287,2061287,2061287,2061287,N,2061287,22.60,0.0,0.5,4.92,0.0,1.0,32.27
4,2743572,1,2743572,2743572,2743572,2743572,None,2743572,18.08,0.0,0.5,0.00,0.0,1.0,22.83


In [ ]:
# Filter by payment_type_id = 1 (Credit card)
payment_type_filter_query = "SELECT * FROM fact_table WHERE payment_type_id = 1;"
payment_type_filtered_df = pd.read_sql_query(payment_type_filter_query, conn)
display(payment_type_filtered_df.head())



,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2,1,1,1,1,N,1,21.2,6.0,0.5,0.0,0.0,1.0,30.45


In [ ]:
# Filter by total_amount > 50
total_amount_filter_query = "SELECT * FROM fact_table WHERE total_amount > 50;"
total_amount_filtered_df = pd.read_sql_query(total_amount_filter_query, conn)
display(total_amount_filtered_df.head())



,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,33.8,6.0,0.5,5.00,6.94,1.0,57.49
1,11,2,11,11,11,11,N,11,38.7,6.0,0.5,11.63,6.94,1.0,69.77
2,25,2,25,25,25,25,N,25,40.1,6.0,0.5,9.52,0.00,1.0,58.87
3,28,2,28,28,28,28,N,28,42.9,6.0,0.5,11.97,6.94,1.0,73.56
4,32,2,32,32,32,32,N,32,52.0,1.0,0.5,7.92,3.18,1.0,68.85


In [ ]:
# Filter by passenger_count_id > 2
passenger_count_filter_query = "SELECT * FROM fact_table WHERE \
passenger_count_id > 2;"
passenger_count_filtered_df = pd.read_sql_query(passenger_count_filter_query, conn)
display(passenger_count_filtered_df.head())



,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,3,7,3,3,3,3,N,3,11.4,0.0,0.5,3.43,0.0,1.0,20.58
1,4,2,4,4,4,4,N,4,11.4,1.0,0.5,2.57,0.0,1.0,19.72
2,5,2,5,5,5,5,N,5,18.4,1.0,0.5,0.00,0.0,1.0,24.15
3,6,2,6,6,6,6,N,6,24.7,1.0,0.5,1.00,0.0,1.0,31.45
4,7,2,7,7,7,7,N,7,7.9,1.0,0.5,2.73,0.0,1.0,16.38


In [ ]:
#close the connection to the SQLite database
conn.close()

